In [1]:
from nanoHUB.clustering.infra import db_query_active_users_by_semester, db_query_active_users_by_month
from nanoHUB.clustering.infra import get_active_users_by_month, save_active_users_by_month, get_active_users_by_semester, save_active_users_by_semester, save_active_users_by_year, get_active_users_by_year, save_active_users_by_academic_year, get_active_users_by_academic_year 
from nanoHUB.application import Application
from nanoHUB.configuration import ClusteringConfiguration
from nanoHUB.pandas_helper import remove_display_restrictions
from nanoHUB.pipeline.geddes.data import get_default_s3_client
from nanoHUB.dataaccess.lake import S3FileMapper
import pandas as pd


remove_display_restrictions(pd)

application = Application.get_instance()
nanohub_metrics_db = application.new_db_engine('nanohub_metrics')
s3_client = get_default_s3_client(application)
mapper = S3FileMapper(s3_client, ClusteringConfiguration().bucket_name_processed)

nanoHUB - Serving Students, Researchers & Instructors


In [2]:
sql_query = '''
SELECT  YEAR(starts1.datetime)
        , MONTH(starts1.datetime)
        , COUNT(DISTINCT starts2.user) num_active_users
  FROM nanohub_metrics.toolstart starts1
  JOIN nanohub_metrics.toolstart starts2
    ON starts2.datetime BETWEEN starts1.datetime - INTERVAL 1 YEAR AND starts1.datetime
  WHERE 
     YEAR(starts1.datetime) > 2008 AND YEAR(starts1.datetime) < 2010 AND
     YEAR(starts2.datetime) < 2010
 GROUP 
    BY YEAR(starts1.datetime), MONTH(starts1.datetime)
;
'''

rolling_df = pd.read_sql_query(sql_query, nanohub_metrics_db)
display(rolling_df)

OperationalError: (pymysql.err.OperationalError) (2013, 'Lost connection to MySQL server during query')
[SQL: 
SELECT  YEAR(starts1.datetime)
        , MONTH(starts1.datetime)
        , COUNT(DISTINCT starts2.user) num_active_users
  FROM nanohub_metrics.toolstart starts1
  JOIN nanohub_metrics.toolstart starts2
    ON starts2.datetime BETWEEN starts1.datetime - INTERVAL 1 YEAR AND starts1.datetime
  WHERE 
     YEAR(starts1.datetime) > 2008 AND YEAR(starts1.datetime) < 2010 AND
     YEAR(starts2.datetime) < 2010
 GROUP 
    BY YEAR(starts1.datetime), MONTH(starts1.datetime)
;
]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
sql_query = '''
    SELECT starts1.datetime
            , starts2.num_active_users AS num_active_users
            , (
                SELECT 
                    starts2.datetime
                    , COUNT(DISTINCT starts2.user) AS num_active_users
                FROM nanohub_metrics.toolstart starts2
                WHERE 
                    starts2.datetime BETWEEN starts1.datetime - INTERVAL 1 YEAR AND starts1.datetime
            ) 
        FROM nanohub_metrics.toolstart starts1
        WHERE 
            YEAR(starts1.datetime) > 2008 AND YEAR(starts1.datetime) < 2010
        GROUP 
            BY YEAR(starts1.datetime), MONTH(starts1.datetime)
    ;
'''

In [ ]:
raise

In [ ]:
sql_query = '''
SELECT YEAR(datetime), MONTH(datetime), YEAR(starts1.datetime - INTERVAL 1 YEAR) AS FROM_YEAR, 
    COUNT(DISTINCT starts1.user),
       (
           SELECT (COUNT(DISTINCT starts2.user))
            FROM nanohub_metrics.toolstart AS starts2
            WHERE
                starts2.datetime >= ((starts1.DATETIME) + INTERVAL (1 - DAY((starts1.DATETIME))) DAY) - INTERVAL 12 MONTH AND
                  starts2.datetime < ((starts1.DATETIME) + INTERVAL (1 - DAY((starts1.DATETIME))) DAY) + INTERVAL 1 MONTH  

                   ) as ROLLING_SUM_OF_ACTIVE_USERS
FROM nanohub_metrics.toolstart AS starts1
WHERE YEAR(datetime) > 2008 AND YEAR(datetime) < 2010
GROUP BY YEAR(datetime), MONTH(datetime)
;
'''
rolling_df = pd.read_sql_query(sql_query, nanohub_metrics_db)
display(rolling_df)

In [ ]:
sql_query = '''
SELECT YEAR(datetime) AS YEAR, MONTH(datetime) AS MONTH, COUNT(DISTINCT starts1.user) AS ACTIVE_USERS,
       (SELECT COUNT(DISTINCT starts2.user)
        FROM nanohub_metrics.toolstart AS starts2
        WHERE YEAR(starts2.datetime) < YEAR(starts1.datetime - INTERVAL 1 YEAR) OR
              (YEAR(starts2.datetime) = YEAR(starts1.datetime) AND MONTH(starts2.datetime) <= MONTH(starts1.datetime))
       ) as 12MONTH_ROLLING_SUM_OF_ACTIVE_USERS
FROM nanohub_metrics.toolstart AS starts1
WHERE YEAR(datetime) > 2007
GROUP BY YEAR(datetime), MONTH(datetime)
;
'''
rolling_df = pd.read_sql_query(sql_query, nanohub_metrics_db)
display(rolling_df)

In [ ]:
save_active_users_by_year(application)

In [ ]:
active_users = get_active_users_by_year(mapper)
print(active_users)

In [ ]:
save_active_users_by_academic_year(application)

In [ ]:
active_users = get_active_users_by_academic_year(mapper)
print(active_users)

In [ ]:
save_active_users_by_month(application)

In [ ]:
active_users = get_active_users_by_month(mapper)
print(active_users)

In [ ]:
save_active_users_by_semester(application)

In [ ]:
active_users = get_active_users_by_semester(mapper)
print(active_users)